## Download CHL Data

In [ ]:
!mkdir -p chl_data

for y in range(1997, 2000):
    !wget -e robots=off -nd -r -np -l1 \
          -A "S{y}*_chl_comp.hdf" \
          https://spg-satdata.ucsd.edu/{y}/S{y}_chl_day/ \
          -P chl_data

In [ ]:
for y in range(2000, 2008):
    !wget -e robots=off -nd -r -np -l1 \
          -A "C{y}*_chl_comp.hdf" \
          https://spg-satdata.ucsd.edu/{y}/C{y}_chl_day/ \
          -P chl_data

In [ ]:
from pyhdf.SD import SD, SDC

# Example file
file_path = "/Users/deliacarpenter/Desktop/Research/primary-productivity/data/chl_data/C2000001_chl_comp.hdf"

# Open the HDF file
hdf = SD(file_path, SDC.READ)

# List all datasets
print("Datasets in HDF file:", hdf.datasets())

In [ ]:
chl_data = hdf.select('chl_2000001')[:]  # shape (rows, cols)
print("Shape of CHL array:", chl_data.shape)
print("Min/max CHL values:", chl_data.min(), chl_data.max())

In [ ]:
sds = hdf.select('chl_2000001')
print(sds.attributes())

In [ ]:
hdf.attributes()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyhdf.SD import SD, SDC

# --- Open your HDF file ---
hdf_file = "/Users/deliacarpenter/Desktop/Research/primary-productivity/data/chl_data/C2000001_chl_comp.hdf"
hdf = SD(hdf_file, SDC.READ)

# --- Select dataset ---
sds_name = list(hdf.datasets().keys())[0]  # 'chl_1997335_1997365'
sds = hdf.select(sds_name)
chl_data = sds[:]  # shape (3405, 3840)

# --- Handle fill values ---
fill_value = sds.attributes()['_FillValue']
chl_data = chl_data.astype(np.float32)
chl_data[chl_data == fill_value] = np.nan  # mask missing values

# --- Apply scaling (from attributes) ---
slope = sds.attributes()['Slope']
intercept = sds.attributes()['Intercept']
base = sds.attributes()['Base']

# Original equation: Base ** (Slope * data + Intercept)
chl_data_scaled = base ** (slope * chl_data + intercept)

# --- Approximate lat/lon grid ---
nrows, ncols = chl_data.shape
min_lat, max_lat = 16, 45
min_lon, max_lon = -135, -100

lats = np.linspace(max_lat, min_lat, nrows)
lons = np.linspace(min_lon, max_lon, ncols)
lon_grid, lat_grid = np.meshgrid(lons, lats)

# --- Plot ---
plt.figure(figsize=(12,6))
plt.pcolormesh(lon_grid, lat_grid, chl_data_scaled, shading='auto', cmap='viridis')
plt.colorbar(label='Chlorophyll (mg m^-3)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('SeaWiFS CHL Composite 2000 001')
plt.show()